## 🚦 การสร้างคอลัมน์เงื่อนไขด้วย `np.where`
---

ในงานจัดการข้อมูล เรามักจะต้องสร้างคอลัมน์ใหม่ที่ระบุสถานะตามเงื่อนไข (เช่น ถ้าวันเหลือน้อยกว่า 0 ให้แสดงว่า "หมดอายุ") เราจะใช้คำสั่ง `np.where` จาก Library **NumPy** ซึ่งทำงานได้รวดเร็วมากครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Conditional Logic** | คอน-ดิ-ชัน-นอล ลอ-จิก | **ตรรกะแบบมีเงื่อนไข** (ถ้า...จริง ให้ทำ A, ถ้าไม่จริง ให้ทำ B) |
| **Vectorized Operation** | เวก-เตอ-ไรซ์ ออป-เปอ-เร-ชัน | การคำนวณที่ทำกับข้อมูลทั้งชุดพร้อมกัน (ไม่ต้องวนลูปเอง) |
| **Binary Choice** | ไบ-นา-รี ช้อยส์ | การเลือกทางใดทางหนึ่งจาก 2 ทางเลือก |
| **Status Label** | สเต-ตัส เล-เบิล | ป้ายระบุสถานะ (เช่น "ปกติ", "เตือน", "ยกเลิก") |

### 💡 รูปแบบการใช้ `np.where`
`np.where(เงื่อนไข, ค่าถ้าจริง, ค่าถ้าเท็จ)`

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

excel_obj = pd.ExcelFile('../Lesson_1/mock_isp_data.xlsx')
sheets_name = excel_obj.sheet_names
print(sheets_name)

df = excel_obj.parse('Main_Data')

df['MSISDN'] = df['MSISDN'].apply(str)

date_cols = [
    'CONTRACT_START_DT',
    'CONTRACT_END_DT'
]

df[date_cols] = df[date_cols].apply(pd.to_datetime)

today = pd.Timestamp(datetime.now().date())

# คำนวณวันคงเหลือสัญญา
df['REMAIN_DAYS'] = (df['CONTRACT_END_DT'] - today).dt.days

# มูลค่าสัญญา
df['REMAIN_VALUE'] = df['REMAIN_DAYS'] * (df['RC_RATE'] / 30)


df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'REMAIN_VALUE']]

['Main_Data']


,CUST_FULL_NAME,REMAIN_DAYS,REMAIN_VALUE
0,นายหมดอายุ แล้วครับ,-49,-978.366667
1,บริษัท กำลังจะหมด (15วัน),9,269.700000
2,ร้านค้า ยังเหลือเยอะ,316,12640.000000
3,นายพอดีเป๊ะ (วันนี้หมด),-6,-89.800000
4,บจก. เพิ่งเริ่มสัญญา,681,6787.300000


### 🧬 การแบ่งกลุ่มที่ซับซ้อนขึ้นด้วย `np.select`
บางครั้ง 2 ทางเลือก (จริง/เท็จ) อาจไม่พอสำหรับงาน ISP ของป๋า เช่น ป๋าอยากแบ่งเป็น:
1. **Expired**: วันติดลบ
2. **Warning**: เหลือไม่เกิน 30 วัน
3. **Active**: ยังเหลืออีกเยอะ

เราจะใช้ `np.select` เพื่อทำ **"Mapping"** เงื่อนไขหลาย ๆ ชั้นครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Multi-Condition** | มัล-ติ คอน-ดิ-ชัน | เงื่อนไขหลายชั้น |
| **Mapping** | แมพ-พิง | การจับคู่เงื่อนไขกับผลลัพธ์ที่ต้องการ |
| **Criteria** | ไค-ที-เรีย | เกณฑ์การตัดสินใจ |
| **Default Value** | ดี-ฟอลต์ แว-ลู | ค่าเริ่มต้น (กรณีที่ไม่ตรงกับเงื่อนไขใดเลย) |

In [11]:
# 1. กำหนดรายการเงื่อนไข (Conditions)
conditions = [
    (df['REMAIN_DAYS'] < 0),                          # กลุ่มที่ 1: หมดอายุ
    (df['REMAIN_DAYS'] >= 0) & (df['REMAIN_DAYS'] <= 30), # กลุ่มที่ 2: ใกล้หมด
    (df['REMAIN_DAYS'] > 30)                           # กลุ่มที่ 3: ปกติ
]

# 2. กำหนดชื่อกลุ่ม (Choices) ให้ตรงตามลำดับเงื่อนไข
choices = ['EXPIRED', 'WARNING', 'HEALTHY']

# 3. ใช้ np.select เพื่อสร้างคอลัมน์ 'CONTRACT_STATUS'
# default='UNKNOWN' คือถ้าหลุดจากทุกเงื่อนไขให้ใส่คำนี้
df['CONTRACT_STATUS'] = np.select(conditions, choices, default='UNKNOWN')

# 4. แสดงผลลัพธ์แบบเรียงลำดับสถานะ
display(df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'CONTRACT_STATUS']].sort_values('REMAIN_DAYS'))

,CUST_FULL_NAME,REMAIN_DAYS,CONTRACT_STATUS
0,นายหมดอายุ แล้วครับ,-49,EXPIRED
3,นายพอดีเป๊ะ (วันนี้หมด),-6,EXPIRED
1,บริษัท กำลังจะหมด (15วัน),9,WARNING
2,ร้านค้า ยังเหลือเยอะ,316,HEALTHY
4,บจก. เพิ่งเริ่มสัญญา,681,HEALTHY


# 📊 บทที่ 3: การสรุปผลข้อมูล (Data Aggregation)
---

ในงาน IT Support หรือ Network Engineer เวลาป๋าจะส่งรายงานให้หัวหน้า เขาไม่ดูหรอกครับว่าลูกค้าคนไหนชื่ออะไร แต่เขาจะถามว่า **"แต่ละกลุ่มมีกี่คน?"** หรือ **"ยอดเงินรวมแต่ละประเภทเป็นเท่าไหร่?"** เราจะใช้คำสั่ง `groupby` ในการหาคำตอบนี้ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Aggregation** | แอ็ก-เกร-เก-ชัน | **การรวบรวมข้อมูล** หลาย ๆ ชิ้นมาสรุปเป็นค่าเดียว (เช่น ผลรวม, ค่าเฉลี่ย) |
| **Groupby** | กรุ๊ป-บาย | **การจัดกลุ่ม** ข้อมูลตามหัวข้อที่สนใจ (เหมือนการคัดแยกประเภทสินค้า) |
| **Count** | เคาต์ | การนับจำนวนรายการ |
| **Sum** | ซัม | การหาผลรวมของตัวเลข |
| **Pivot Table** | พิ-วอท เท-เบิล | ตารางสรุปผลที่มีการตัดขวางข้อมูลหลายมิติ |

> **หัวใจของบทนี้:** "จัดกลุ่มตามอะไร (By) และจะให้ทำอะไรกับตัวเลข (Action)"

---

💻 ส่วนที่ 2: โค้ดการนับจำนวนตามสถานะ (Code Cell)
ป๋าลองดูว่าหลังจากที่เราแบ่งกลุ่ม CONTRACT_STATUS ไปแล้ว แต่ละกลุ่มมีลูกค้ากี่รายครับ

In [12]:
# 1. จัดกลุ่มตามสถานะสัญญา (CONTRACT_STATUS) แล้วนับจำนวน (size)
status_summary = df.groupby('CONTRACT_STATUS').size()

# 2. แสดงผล (จะออกมาเป็น Series)
print("สรุปจำนวนลูกค้าตามสถานะ:")
display(status_summary)

# 3. ถ้าอยากให้เป็น DataFrame สวยๆ ให้เติม .reset_index(name='COUNT')
status_df = status_summary.reset_index(name='CUSTOMER_COUNT')
display(status_df)

สรุปจำนวนลูกค้าตามสถานะ:


CONTRACT_STATUS
EXPIRED    2
HEALTHY    2
WARNING    1
dtype: int64

,CONTRACT_STATUS,CUSTOMER_COUNT
0,EXPIRED,2
1,HEALTHY,2
2,WARNING,1


### 📈 การทำรายงานสรุปแบบหลายมิติ (.agg)
บางทีป๋าอาจจะอยากรู้ทั้ง **"จำนวนคน"** และ **"ยอดเงินรวม (RC_RATE)"** ไปพร้อม ๆ กัน เราจะใช้ฟังก์ชัน `.agg()` (ย่อมาจาก Aggregate) ซึ่งป๋าสามารถสั่งงานได้หลายอย่างในคำสั่งเดียวครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Mean** | มีน | ค่าเฉลี่ย |
| **Summary Table** | ซัม-มา-รี เท-เบิล | ตารางสรุปผลลัพธ์สุดท้าย |
| **Multi-Aggregation** | มัล-ติ แอ็ก-เกร-เก-ชัน | การคำนวณหลาย ๆ อย่างพร้อมกัน (เช่น ทั้งนับทั้งบวก) |


---

💻 ส่วนที่ 4: โค้ดสร้างรายงานสรุปส่งหัวหน้า (Code Cell)
โค้ดชุดนี้จะสรุปให้ป๋าเลยว่า แต่ละสถานะมีกี่คน และเงินหายไปเท่าไหร่ (Revenue at Risk)

In [ ]:
# 1. จัดกลุ่มตามสถานะ และสั่งคำนวณ RC_RATE ทั้ง 'count' และ 'sum'
report = df.groupby('CONTRACT_STATUS')['RC_RATE'].agg(['count', 'sum', 'mean'])

# 2. เปลี่ยนชื่อคอลัมน์ให้สื่อสารง่ายขึ้น (Rename)
report = report.rename(columns={
    'count': 'จำนวนลูกค้า (ราย)',
    'sum': 'ยอดเงินรวม (บาท)',
    'mean': 'เฉลี่ยต่อหัว (บาท)'
})

# 3. ตกแต่งให้สวยงาม (ใส่เครื่องหมายคอมม่า และทศนิยม 2 ตำแหน่ง)
styled_report = report.style.format({
    'ยอดเงินรวม (บาท)': '{:,.2f}',
    'เฉลี่ยต่อหัว (บาท)': '{:,.2f}'
})

display(styled_report)

,จำนวนลูกค้า (ราย),ยอดเงินรวม (บาท),เฉลี่ยต่อหัว (บาท)
CONTRACT_STATUS,,,
EXPIRED,2,"1,048.00",524.00
HEALTHY,2,"1,499.00",749.50
WARNING,1,899.00,899.00


## 💾 การส่งออกข้อมูล (Exporting Results)
---

เมื่อป๋าสรุปผลเสร็จแล้ว ขั้นตอนสุดท้ายคือการเซฟกลับเป็นไฟล์ Excel เพื่อส่งอีเมลให้ทีมที่เกี่ยวข้องครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Export** | เอ็กซ์-พอร์ต | การส่งข้อมูลออกไปยังไฟล์ภายนอก |
| **Index** | อิน-เด็กซ์ | ดัชนีหน้าแถว (ถ้าไม่เอาให้ตั้ง index=False) |
| **Encoding** | เอน-โค้ด-ดิง | รหัสภาษา (เช่น utf-8-sig เพื่อให้ภาษาไทยไม่เพี้ยน) |

In [17]:
# บันทึกรายงานสรุปเข้าไฟล์ใหม่
report.to_excel('isp_summary_report.xlsx')

# หรือถ้าป๋าอยากบันทึกข้อมูลดิบที่คำนวณแล้ว (df) ทั้งหมด
df.to_excel('isp_processed_data.xlsx', index=False)

print("ป๋าครับ! บันทึกไฟล์เรียบร้อย พร้อมส่งอีเมลแล้วครับ")

ป๋าครับ! บันทึกไฟล์เรียบร้อย พร้อมส่งอีเมลแล้วครับ
